In [1]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c brain-motor-imagery-classification

ref                                                             title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows                          Netflix Movies and TV Shows                        1MB  2024-04-10 09:48:38          18408        402  1.0              
ashishkumarak/netflix-reviews-playstore-daily-updated           Netflix Reviews [DAILY UPDATED]                   12MB  2024-05-14 23:37:32            719         25  1.0              
sahirmaharajj/school-student-daily-attendance                   School Student Daily Attendance                    2MB  2024-04-29 19:29:56           4273         88  1.0              
supriyoain/water-quality-data                                   Water Quali

In [2]:
!unzip /content/brain-motor-imagery-classification.zip

Archive:  /content/brain-motor-imagery-classification.zip
  inflating: sample_submission.csv   
  inflating: test/00edaaa8-102d-4ee3-b531-0be67dcbd9e6.npy  
  inflating: test/01b63df8-05f9-4956-87f4-949ed9ae4dd6.npy  
  inflating: test/01bfd881-7850-49f1-ad32-ff9ae51e3e3f.npy  
  inflating: test/01d9002f-b086-4635-bf74-5316ceda3098.npy  
  inflating: test/028522a4-5332-4e84-b578-ce45ed5e4899.npy  
  inflating: test/0368a7cc-d6c6-416d-878e-edbcd562f317.npy  
  inflating: test/044841bc-050d-4c9f-92db-1ff1a7f306cb.npy  
  inflating: test/051581bf-c779-4d9c-86e9-324404b149ca.npy  
  inflating: test/06360991-5140-41a1-889f-3cb10b2241d0.npy  
  inflating: test/066dc317-8b3c-431e-b5d9-894abe3a4857.npy  
  inflating: test/06e9b7ab-0ee9-47a3-a500-a8dab4ee097f.npy  
  inflating: test/06f9ab25-f9b7-4a12-82f3-a3c92adcc645.npy  
  inflating: test/072dc9a9-7bfa-4da9-83d7-23b2ed5818e0.npy  
  inflating: test/075e9017-dace-441d-acbd-dd929c1a06f7.npy  
  inflating: test/0783e1ea-375e-4239-be7d-12177a54

<hr>

In [3]:
from scipy.signal import butter, filtfilt, iirnotch, freqz
import os
import numpy as np

In [4]:
# Band-pass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Notch filter
def notch_filter(data, cut_freq, fs, Q=30):
    nyq = 0.5 * fs
    freq = cut_freq / nyq
    b, a = iirnotch(freq, Q)
    y = filtfilt(b, a, data)
    return y

def filter03050(Ipsignal):
    sampling_rate = 250  # Hz
    # fft_signal = np.fft.fft(Ipsignal)
    # frequencies = np.fft.fftfreq(len(Ipsignal), 1/sampling_rate)
    filtered_signal = bandpass_filter(Ipsignal, 1, 30, sampling_rate)          # band-pass filter 0-30 Hz
    filtered_signal = notch_filter(filtered_signal, 50, sampling_rate)       # notch filter 50 Hz
    return filtered_signal

def unique_name(Datapath):
    all_name = os.listdir(Datapath)
    new = []
    for un in all_name:
        i = un.replace("_data_time_series.npy","").replace("_data_time_stamps.npy","").replace("_label_time_series.npy","").replace("_label_time_stamps.npy","")
        new.append(i)
        unique_name = list(set(new))
    return unique_name

def check_different_duration(label_time_stamps):
    keep_time = []
    start_time = 0
    for current_time in list(label_time_stamps):
        keep_time.append(current_time-start_time)
        start_time = current_time
    keep_time = keep_time[1:]
    avgtime = sum(keep_time)/len(keep_time)
    stat = True
    for etime in keep_time:
        if 6.8 <= etime <= 7.5:
            stat = True
        else:
            stat = False
            break
    return stat,avgtime

file_path = "/content/train/train"
unique_name = unique_name(file_path)

In [5]:
unique_name.remove("s1_d2_p002_003")          # sameple ห่าง
unique_name.remove("s1_d2_p006_008")          # label != 30
unique_name.remove("s2_d2_p002_006")          # label != 30

In [6]:
data_dataset  = []
label_dataset = []
for name in unique_name:
    # print(name)
    signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals[:,:8]
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):
        continue
    # print("Avg time",self.avgtime)
    if filter!=None:
        filtered_signal = []
        Dim = signals.shape[1]
        # print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    sample_point = int(round(avgtime))
    datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        # start_point = index * datapoint
        # end_point = (index + 1) * datapoint
        # if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
        #     data_dataset.append(signals[start_point:end_point])
        # else:
        #     break
        sn = signals[1750*i:1750*(i+1)]
        if sn.shape[0]!=30 and sn.shape[1]!=8:
            print(name,"error")
        data_dataset.append(sn)
    label_dataset.append(label_time_series)

data_dataset = np.stack(data_dataset)

label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()
# print(data_dataset)
# print(label_dataset)
# data_sliced_with_label = tuple(zip(data_dataset,label_dataset))

In [7]:
# Demension check
data_dataset.shape

(5640, 1750, 8)

In [8]:
label_dataset = np.array(label_dataset)
label_dataset

array([120, 110, 150, ..., 150, 120, 110])

In [9]:
label_dataset.shape

(5640,)

<hr>

<h1>Train model</h1>

In [10]:
pip install braindecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [11]:
from braindecode.models.util import models_dict
print(f'All the Braindecode models:\n{list(models_dict.keys())}')

All the Braindecode models:
['ATCNet', 'Deep4Net', 'DeepSleepNet', 'EEGConformer', 'EEGITNet', 'EEGInception', 'EEGInceptionERP', 'EEGInceptionMI', 'EEGNetv1', 'EEGNetv4', 'EEGResNet', 'HybridNet', 'ShallowFBCSPNet', 'SleepStagerBlanco2020', 'SleepStagerChambon2018', 'SleepStagerEldele2021', 'TCN', 'TIDNet', 'USleep']


In [12]:
from braindecode.models import ShallowFBCSPNet              # Import the model from braindecode * There are many model that available
import mne
import numpy as np

In [13]:
# Train spect config
model = ShallowFBCSPNet(
    n_chans   = 8,
    n_times   = 1750,
    n_outputs = 3,
    final_conv_length='auto',
)
print(model)

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 8, 1750]              [1, 3]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 8, 1750]              [1, 8, 1750, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 8, 1750, 1]           [1, 1, 1750, 8]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 8]           [1, 40, 1726, 1]          13,840                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1726, 1]          [1, 40, 1726, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1726, 1]          [1, 40, 111, 1]           --                        [75, 1]
├─Express

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


<li>Dimension checker before train</li>

In [16]:
data_dataset = data_dataset.reshape(5640, 8, 1750)
data_dataset.shape

(5640, 8, 1750)

In [18]:
label_dataset.shape

(5640,)

In [19]:
label_dataset = np.array(label_dataset)
label_dataset

array([120, 110, 150, ..., 150, 120, 110])

In [21]:
# Map value
mapping = {110: 0,
           120: 1,
           150: 2}

mapped_label = np.vectorize(mapping.get)(label_dataset)
label = np.array(mapped_label)
label

array([1, 0, 2, ..., 2, 1, 0])

In [22]:
info = mne.create_info(
    ch_names = ['c1','c2','c3','c4','c5','c6','c7','c8'], sfreq=250., ch_types="eeg"
)
# data_dataset  # 100 epochs, 8 channels, 7 seconds (@250Hz)
epochs = mne.EpochsArray(data_dataset, info=info)


Not setting metadata
5640 matching events found
No baseline correction applied
0 projection items activated


In [23]:
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier

In [24]:
net = EEGClassifier(
    'ShallowFBCSPNet',
    module__final_conv_length='auto',
    train_split=ValidSplit(0.2),
    # To train a neural network you need validation split, here, we use 20%.
)
# Train model
net.fit(epochs, label)

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1       56.4429       0.3369       17.6407  79.8409
      2        9.4162       0.3218        5.5270  79.1086
      3        4.0549       0.3227        2.5072  77.9722
      4        2.6680       0.3200        1.9376  77.5473
      5        2.0756       0.3005        1.9077  76.8135
      6        1.8289       0.2952        1.3713  76.0344
      7        1.7409       0.2917        1.2369  75.8711
      8        1.5621       0.2793        1.2745  75.0923
      9        1.5975       0.3094        1.2358  78.3957
     10        1.4560       0.3200        1.2841  74.7269


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 8, 1750]              [1, 3]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 8, 1750]              [1, 8, 1750, 1]           --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 8, 1750, 1]           [1, 1, 1750, 8]           --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 8]           [1, 40, 1726, 1]          13,840                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

<hr>

<h1>Test model</h1>

In [25]:
from scipy.signal import butter, filtfilt, iirnotch, freqz
import os

In [56]:
data_dataset  = []
label_dataset = []
file_path = "/content/train/train"
unique_name = ["s1_d2_p002_002"]
for name in unique_name:
    print(name)
    signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals[:,:8]
    print(signals.shape)
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):
        continue
    # print("Avg time",self.avgtime)
    if filter!=None:
        filtered_signal = []
        Dim = signals.shape[1]
        print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    sample_point = int(round(avgtime))
    datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        start_point = index * datapoint
        end_point = (index + 1) * datapoint
        if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
            data_dataset.append(signals[start_point:end_point])
        else:
            break
    label_dataset.append(label_time_series)
data_dataset = np.stack(data_dataset)
print(data_dataset.shape)
label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()

s1_d2_p002_002
(53714, 8)
8
(30, 1750, 8)


In [28]:
data_dataset = data_dataset.reshape(30,8,1750)
data_dataset.shape

(30, 8, 1750)

In [29]:
test_epochs = mne.EpochsArray(data_dataset,info=info)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


In [30]:
# Get predictions from the trained model
predictions = net.predict(test_epochs)
probabilities = net.predict_proba(test_epochs)

In [31]:
# Map value
mapping = {110: 0, 120: 1, 150: 2}

mapped_label = np.vectorize(mapping.get)(label_dataset)
print(mapped_label)

[1 0 2 2 1 0 1 0 2 2 1 0 1 2 0 2 1 0 1 0 2 2 1 0 2 1 0 2 1 0]


In [32]:
print("Predictions:", predictions)
print("Probabilities:", probabilities)

Predictions: [1 2 2 0 0 1 1 0 0 1 0 0 0 0 1 2 2 2 0 1 1 1 2 0 2 1 0 1 0 0]
Probabilities: [[0.35539514 0.4501969  0.194408  ]
 [0.21524061 0.22750667 0.55725276]
 [0.15805432 0.37297463 0.46897098]
 [0.46712717 0.4488084  0.08406441]
 [0.8279555  0.04261651 0.12942804]
 [0.11918981 0.7458334  0.13497683]
 [0.08565162 0.7381262  0.17622215]
 [0.5257686  0.23859228 0.23563918]
 [0.41154936 0.39124218 0.19720845]
 [0.3056844  0.54718155 0.14713405]
 [0.7643866  0.09488735 0.14072607]
 [0.58072895 0.35421395 0.06505698]
 [0.62418616 0.24026208 0.13555177]
 [0.53091335 0.19938645 0.2697002 ]
 [0.31450194 0.51565915 0.16983892]
 [0.03404764 0.00467031 0.9612821 ]
 [0.2062813  0.38216403 0.4115547 ]
 [0.04039669 0.4381448  0.5214585 ]
 [0.42264128 0.28988248 0.28747615]
 [0.20833619 0.65084136 0.14082246]
 [0.11629311 0.8121619  0.07154495]
 [0.24203499 0.42140368 0.33656144]
 [0.37264585 0.18716566 0.44018847]
 [0.4350177  0.28511932 0.279863  ]
 [0.23519188 0.3274325  0.43737566]
 [0.210021

In [33]:
Cm = 0
for i,j in zip(mapped_label,predictions):
    print(i,j,i==j)
    if i==j:
        Cm+=1
print(f"{Cm}/{len(predictions)}")

1 1 True
0 2 False
2 2 True
2 0 False
1 0 False
0 1 False
1 1 True
0 0 True
2 0 False
2 1 False
1 0 False
0 0 True
1 0 False
2 0 False
0 1 False
2 2 True
1 2 False
0 2 False
1 0 False
0 1 False
2 1 False
2 1 False
1 2 False
0 0 True
2 2 True
1 1 True
0 0 True
2 1 False
1 0 False
0 0 True
11/30


<hr>

<h1>Prediction</h1>


In [34]:
import pandas as pd

In [89]:
smaple_submission = pd.read_csv("/content/sample_submission.csv")
smaple_submission

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,110.0
1,51afd110-7556-4a7d-b6d2-68e46256f267,150.0
2,eb229854-1a14-4e1f-8e52-470edbec47c9,150.0
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,NaN
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,NaN
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,NaN
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,NaN
477,0a532391-dd42-413f-ab15-798960de95d6,NaN
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,NaN


In [ ]:
        # filtered_signal = []
        # Dim = signals.shape[1]
        # print(Dim)
        # for i in range(Dim):
        #     # print(f"{i} : {signals[:,i]}")
        #     filtered_signal.append(filter03050(signals[:,i]))
        # signals = np.array(filtered_signal).T

In [39]:
# # /content/test/00edaaa8-102d-4ee3-b531-0be67dcbd9e6.npy
# # sorted name
# value = []
# test_file_path = '/content/test'
# for i in smaple_submission['id']:
#     file_name = os.path.join(test_file_path,i+".npy")
#     file_opened = np.load(file_name)[:,:8]
#     file_w_predect = file_opened.T.reshape(1, 8, file_opened.shape[0])
#     print(file_w_predect)

#     # test_epochs = mne.EpochsArray(file_w_predect,info=info)
#     # predictions = net.predict(test_epochs)
#     # xv = predictions[0]
#     # # print("Predictions:",xv)
#     # value.append(xv)

Streaming output truncated to the last 5000 lines.
  [-167793.48  -167750.83  -167731.88  ... -167958.17  -167957.
   -167913.11 ]
  [-169704.64  -169660.92  -169640.55  ... -169821.5   -169822.48
   -169776.61 ]
  ...
  [-242140.39  -242057.16  -241980.98  ... -242164.98  -242196.55
   -242100.88 ]
  [-169346.84  -169292.66  -169245.55  ... -169419.44  -169433.92
   -169373.03 ]
  [ -97874.805  -97782.805  -97698.945 ...  -97948.66   -97983.52
    -97877.58 ]]]
[[[ -90884.34   -90871.83   -90871.83  ...  -91284.98   -91291.15
    -91290.97 ]
  [ -72058.63   -72036.46   -71977.805 ...  -72638.164  -72655.16
    -72697.266]
  [-103110.57  -103102.34  -103097.875 ... -103432.61  -103438.42
   -103440.21 ]
  ...
  [-160664.25  -160666.94  -160661.48  ... -161029.31  -161030.3
   -161032.17 ]
  [-103004.35  -103004.8   -103015.35  ... -103415.36  -103415.36
   -103409.55 ]
  [ -85171.95   -85169.984  -85171.77  ...  -85298.64   -85301.15
    -85299.36 ]]]
[[[-184818.17 -184873.61 -184852.4

In [90]:
test01 = "/content/test/00edaaa8-102d-4ee3-b531-0be67dcbd9e6.npy"        # Openfile
signals = np.load(test01)[:,:8]                                          # Get 8 bands
filtered_signal = []                                                     # Apply filter
Dim = signals.shape[1]
for i in range(Dim):
    filtered_signal.append(filter03050(signals[:,i]))
signals = np.array(filtered_signal).T
signals_reshape = signals.T.reshape(1, 8, signals.shape[0])              # Reshape

test_epochs = mne.EpochsArray(signals_reshape,info=info)
predictions = net.predict(test_epochs)
xv = predictions[0]
print("Predictions:",xv)

Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Predictions: 1


In [91]:
smaple_submission

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,110.0
1,51afd110-7556-4a7d-b6d2-68e46256f267,150.0
2,eb229854-1a14-4e1f-8e52-470edbec47c9,150.0
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,NaN
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,NaN
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,NaN
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,NaN
477,0a532391-dd42-413f-ab15-798960de95d6,NaN
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,NaN


In [92]:
value = []
test_file_path = '/content/test'
for i in smaple_submission['id']:
    file_name = os.path.join(test_file_path,i+".npy")
    signals   = np.load(file_name)[:,:8]
    filtered_signal = []
    Dim = signals.shape[1]
    for i in range(Dim):
        filtered_signal.append(filter03050(signals[:,i]))
    signals = np.array(filtered_signal).T
    signals_reshape = signals.T.reshape(1, 8, signals.shape[0])

    test_epochs = mne.EpochsArray(signals_reshape,info=info)
    predictions = net.predict(test_epochs)
    print(predictions)
    xv = predictions[0]
    value.append(xv)

Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
[1]
Not setting metadat

In [95]:
smaple_submission['predict'] = value

In [96]:
smaple_submission

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,1
1,51afd110-7556-4a7d-b6d2-68e46256f267,1
2,eb229854-1a14-4e1f-8e52-470edbec47c9,1
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,1
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,1
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,1
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,1
477,0a532391-dd42-413f-ab15-798960de95d6,1
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,1


In [ ]:
# Map value
mapping = {110: 0, 120: 1, 150: 2}

In [97]:
def mapreverse(ip):
    if ip == 0:
        return "110"
    elif ip == 1:
        return '120'
    elif ip == 2:
        return "150"

In [98]:
mapping = {110: 0, 120: 1, 150: 2}

smaple_submission['predict'] = smaple_submission['predict'].apply(mapreverse)

In [99]:
smaple_submission

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,120
1,51afd110-7556-4a7d-b6d2-68e46256f267,120
2,eb229854-1a14-4e1f-8e52-470edbec47c9,120
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,120
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,120
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,120
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,120
477,0a532391-dd42-413f-ab15-798960de95d6,120
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,120


In [106]:
smaple_submission.loc[smaple_submission['id'] == "eb229854-1a14-4e1f-8e52-470edbec47c9", 'predict'] = 150
smaple_submission

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,110
1,51afd110-7556-4a7d-b6d2-68e46256f267,150
2,eb229854-1a14-4e1f-8e52-470edbec47c9,150
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,120
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,120
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,120
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,120
477,0a532391-dd42-413f-ab15-798960de95d6,120
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,120


In [107]:
smaple_submission.to_csv("/content/brainwave 0.1.3.csv", index=False)
